<a href="https://colab.research.google.com/github/rdrudi/PythonDataAnalysis/blob/master/HabermanKNN/HabermanKNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cancer Survivor Forecasting
## K-Nearest Neighbors
O código implementa um previsor KNN manualmente, definindo todas as funções.  
Dataset utilizado: https://archive.ics.uci.edu/ml/datasets/HabermansSurvival


In [56]:
# imports e sets
import math
import requests
# percentual de amostras para treinamento
RATE = 0.7
# número de vizinhos
K = 11

In [57]:
# função normaliza: normaliza um vetor qualquer entre [0,1]
def normaliza(lista):
    menor = min(lista)
    maior = max(lista)
    dif = maior - menor
    lista = [((i-menor)/dif) for i in lista]

In [58]:
# função normalizaAmostras: normaliza os dados utilizados na predição
def normalizaAmostras(amostras):
    # separando os campos
    idade,ano,nodulos = [],[],[]
    for s in amostras:
        idade.append(s[0])
        ano.append(s[1])
        nodulos.append(s[2])
    #normalizando idade
    normaliza(idade)
    normaliza(ano)
    normaliza(nodulos)
    for i in range(len(amostras)):
        (amostras[i])[0] = idade[i]
        (amostras[i])[1] = ano[i]
        (amostras[i])[2] = nodulos[i]

In [59]:
# função info_dataset: imprime algumas informações sobre os dados
def info_dataset(amostras, verbose=True):
    if verbose:
        print('Total de amostras: %d' % len(amostras))
    rotulo1, rotulo2 = 0, 0
    for amostra in amostras:
        if amostra[-1] == 1:
            rotulo1 += 1
        else:
            rotulo2 += 1
    if verbose:
        print('Total rotulo 1: %d' % rotulo1)
        print('Total rotulo 2: %d' % rotulo2)
    return [len(amostras), rotulo1, rotulo2]

In [60]:
# função dist_euclidiana: calcula a distância euclidiana entre dois vetores de um espaço n-dimensional
def dist_euclidiana(v1, v2):
    dim, soma = len(v1), 0
    # dim - 1 para não pegar o atributo de saída
    for i in range(dim - 1):
        soma += math.pow(v1[i] - v2[i], 2)
    return math.sqrt(soma)

In [61]:
# função knn: a partir de um grupo de treinamento e uma nova amostra, classifica a nova amostra como sendo do grupo 1 ou grupo 2
def knn(treinamento, nova_amostra, K):
    dists, tam_treino = {}, len(treinamento)

    # calcula a distância euclidiana da nova amostra para 
    # todos os outros exemplos do conjunto de treinamento
    for i in range(tam_treino):
        d = dist_euclidiana(treinamento[i], nova_amostra)
        dists[i] = d

    # obtém as chaves (índices) dos k-vizinhos mais próximos
    k_vizinhos = sorted(dists, key=dists.get)[:K]

    # votação majoritária
    qtd_rotulo1, qtd_rotulo2 = 0, 0
    for indice in k_vizinhos:
        if treinamento[indice][-1] == 1:
            qtd_rotulo1 += 1
        else:
            qtd_rotulo2 += 1

    #print(nova_amostra)
    if qtd_rotulo1 > qtd_rotulo2:
        #print('Classe 1')
        return 1
    else:
        #print('Classe 2')
        return 2

In [62]:
# função calculaAcertos: testa o conjunto de amostras e calcula o número de acertos
def calculaAcertos(K):
    acertos = 0
    for amostra in teste:
        classe = knn(treinamento, amostra, K)
        if amostra[-1] == classe:
            acertos += 1
    return acertos

In [64]:
# Programa Principal
amostras = []
# abre o arquivo no github
url = 'https://raw.githubusercontent.com/rdrudi/PythonDataAnalysis/master/HabermanKNN/haberman.data'
page = requests.get(url)
data = page.text.split(sep='\n')
# pula a primeira linha = cabeçalho
data = data[1:]
# lê as amostras e monta o espaço vetorial
for linha in data:
    # obtém os atributos da amostra
    atrib = linha.split(',')
    # adiciona a amostra na lista
    try:
      idade = int(atrib[0])
      ano = int(atrib[1])
      nodulos = int(atrib[2])
      classe = int(atrib[3])
    # se houver erro de conversão - despreza a amostra
    except ValueError:
      pass
    amostras.append([idade,ano,nodulos,classe])

normalizaAmostras(amostras)

# separa o conjunto em treinamento e teste conforme valor da constante RATE
_, rotulo1, rotulo2 = info_dataset(amostras, verbose=False)
treinamento, teste = [], []
total_rotulo1, total_rotulo2 = int(RATE * rotulo1), int(RATE * rotulo2)
qt_rotulo1, qt_rotulo2 = 0, 0
for amostra in amostras:
    if amostra[-1] == 1 and qt_rotulo1 < total_rotulo1:
        treinamento.append(amostra)
        qt_rotulo1 += 1
    elif amostra[-1] == 2 and qt_rotulo2 < total_rotulo2:
        treinamento.append(amostra)
        qt_rotulo2 += 1
    else:
        teste.append(amostra)
        
acertos = 0
acertos = calculaAcertos(K)

print('Total de treinamento: %d' % len(treinamento))
print('Total de testes: %d' % len(teste))
print('Total de acertos: %d' % acertos)
print('Porcentagem de acertos: %.2f%%' % (100 * acertos / len(teste)))

Total de treinamento: 214
Total de testes: 93
Total de acertos: 68
Porcentagem de acertos: 73.12%
